## Suitable Neighbourhoods for Launching a Yoga Studio in Toronto

It is aimed at identifying top neighbourhoods in Toronto, where Yoga Studios are not amongst the top 5 common fitness centres, which provides an opportunity to launch a new commercial Yoga Studio venture.

### Part 01 - Collecting and Cleaning Data

In [69]:
#importing required libraries

import pandas as pd
import requests
import numpy as np

from pandas.io.json import json_normalize

In [70]:
#acquiring the data, and converting it into a data frame. 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
temp = requests.get(url)

df_temp = pd.read_html(temp.content, header=0)[0]

#Ignore Boroughs where value is 'Not assigned'.
df = df_temp[df_temp.Borough != 'Not assigned']

df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df.loc[df['Neighborhood'] == "Not assigned"]

,Postal Code,Borough,Neighborhood


#### No Row found where Neighborhood is Not Assigned.

In [71]:
#more than one neighborhood under the same postal code
df_tor = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(lambda x:', '.join(x))
df_tor = df_tor.reset_index()
df_tor


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [72]:
#printing size of dataframe
print ("The Dataframe df_tor has", df_tor.shape[0], 'rows.')

The Dataframe df_tor has 103 rows.




### Part 02 - Adding Geographical Co-ordinates to the Data Frame

Collecting Lat-Long of the Locations from csv file containing Geospatial Data

In [73]:
df_latlong = pd.read_csv('https://cocl.us/Geospatial_data')
df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge df_latlong with original dataframe df_tor.

In [74]:
df_tor = pd.merge(df_tor, df_latlong, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
df_tor.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part 03 - Analysing Neighbourhoods in Toronto with respect to Fitness Centres

Opting to work only with boroughs that contain the name 'Toronto'

#### Converting to Lat-Long -

In [75]:
from geopy.geocoders import Nominatim #convert to lat-long

address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Lat-Long Coordinates of Toronto city are ', latitude, longitude)

Lat-Long Coordinates of Toronto city are  43.6534817 -79.3839347


#### Create Map of Toronto with Neighborhoods marked - 

In [76]:
import folium # map rendering

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(
        df_tor['Latitude'], 
        df_tor['Longitude'], 
        df_tor['Borough'], 
        df_tor['Neighborhood']):
    
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

#### Filtering Boroughs with names containing 'Toronto' -

In [13]:
df_tor_filter = df_tor [df_tor['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_tor_filter.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


#### Plotting Map again with the filtered boroughs - 

In [77]:
map_tor_filtered = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, borough, neighborhood in zip(
        df_tor_filter['Latitude'], 
        df_tor_filter['Longitude'], 
        df_tor_filter['Borough'], 
        df_tor_filter['Neighborhood']):
    
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor_filtered)  

map_tor_filtered

#### Define Foursquare Credentials -

In [78]:
CLIENT_ID = 'FJBCXA5W2UXEAIVJ0UCCIWG3Y1G40L3CEZQQGIIBVFLPJ54X' 
CLIENT_SECRET = 'TEBAEOPCYDFOGSG5I4FLWI2HC4P0N5JDZWRULZP32O3UVD3F' 
VERSION = '20180605' # Foursquare API version

### Exploring Neighborhoods in Toronto

In [82]:
def getNearbyGyms(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            '4bf58dd8d48988d175941735', #foursquare category for GYMs
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()# ["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [84]:
def getNearbyGyms(names, latitudes, longitudes, radius=1000, LIMIT=500):
    not_found = 0
    print('***Start ', end='')
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(' .', end='')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            "4bf58dd8d48988d175941735", #foursquare category for GYMs
            LIMIT)
            
        try:
            # make the GET request
            results = requests.get(url).json()['response']['venues']
            
            # return only relevant information for each nearby venue
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['name'], 
                v['location']['lat'], 
                v['location']['lng'],  
                v['categories'][0]['name']) for v in results])
        except:
            not_found += 1


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print("\nDone*** with {} venues with incompelete information.".format(not_found))
    return(nearby_venues)

### Locating Venue Category : 'Gyms/Fitness Centres' in various locations

In [99]:
tor_gyms = getNearbyGyms(names=df_tor_filter['Neighborhood'],
                                   latitudes=df_tor_filter['Latitude'],
                                   longitudes=df_tor_filter['Longitude'])

tor_gyms.head(10)

***Start  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
Done*** with 0 venues with incompelete information.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Beaches BJJ,43.672703,-79.289013,Martial Arts Dojo
1,The Beaches,43.676357,-79.293031,Link Integrated Health,43.683174,-79.300078,Gym
2,The Beaches,43.676357,-79.293031,Bold Steps Dance Studio,43.673555,-79.283204,Gym / Fitness Center
3,The Beaches,43.676357,-79.293031,Beaches Rec Centre,43.673659,-79.298634,College Gym
4,The Beaches,43.676357,-79.293031,Beach Pilates,43.671808,-79.292115,Pilates Studio
5,The Beaches,43.676357,-79.293031,Prana Fitness,43.671070,-79.295043,Yoga Studio
6,The Beaches,43.676357,-79.293031,Fitness Works For Women,43.670846,-79.294619,Gym
7,The Beaches,43.676357,-79.293031,Horizon Martial Arts,43.672480,-79.289242,Martial Arts Dojo
8,The Beaches,43.676357,-79.293031,Glanville Mediation Services,43.678463,-79.299467,Yoga Studio
9,The Beaches,43.676357,-79.293031,Classical Martial Arts Centre CMAC Beaches Dojo,43.669539,-79.301278,Martial Arts Dojo


Check size of dataframe - 

In [87]:
print(tor_gyms.shape)

(1401, 7)


Check number of unique categories - 

In [88]:
print('There are {} uniques categories.'.format(len(tor_gyms['Venue Category'].unique())))

There are 24 uniques categories.


### Analysing all the neighborhoods

In [89]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_gyms[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_gyms['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head(10)

,Neighborhood,Athletics & Sports,Bike Shop,Boxing Gym,Climbing Gym,College Gym,College Rec Center,College Stadium,Cycle Studio,Event Space,...,Martial Arts Dojo,Office,Pilates Studio,Playground,Pool,Recreation Center,Sports Club,Track,Track Stadium,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
8,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [97]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Athletics & Sports,Bike Shop,Boxing Gym,Climbing Gym,College Gym,College Rec Center,College Stadium,Cycle Studio,Event Space,...,Martial Arts Dojo,Office,Pilates Studio,Playground,Pool,Recreation Center,Sports Club,Track,Track Stadium,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040816
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.022727,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.159091
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.037037,0.074074,0.000000,0.000000
4,Central Bay Street,0.000000,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.122449
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.022727,0.000000,...,0.204545,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.250000
6,Church and Wellesley,0.000000,0.020833,0.000000,0.000000,0.000000,0.020833,0.000000,0.020833,0.000000,...,0.000000,0.020833,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.020408
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.045455,0.022727,0.022727,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.136364
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,...,0.023256,0.023256,0.046512,0.000000,0.000000,0.000000,0.000000,0.023256,0.000000,0.162791


### Print Neighborhoods with 5 Most Common Fitness Centres

In [106]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                  venue  freq
0                   Gym  0.61
1  Gym / Fitness Center  0.29
2              Gym Pool  0.04
3           Yoga Studio  0.04
4        Pilates Studio  0.02


----Brockton, Parkdale Village, Exhibition Place----
                  venue  freq
0                   Gym  0.41
1  Gym / Fitness Center  0.34
2           Yoga Studio  0.16
3     Martial Arts Dojo  0.05
4            Boxing Gym  0.02


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Gym / Fitness Center  0.44
1                   Gym  0.31
2           Yoga Studio  0.25
3        Gymnastics Gym  0.00
4         Track Stadium  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0                   Gym  0.44
1  Gym / Fitness Center  0.41
2                 Track  0.07
3           Sports Club  0.04
4        Pilates Studio

### Creating a PD Dataframe for 5 Most Common Fitness Centres

In [107]:
def return_most_common_venues(row, num_top_gyms):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Creating DF with 5 Most Common Fitness Centres

In [118]:
num_top_gyms = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Fitness Centre'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Fitness Centre'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Fitness Centre,2nd Most Common Fitness Centre,3rd Most Common Fitness Centre,4th Most Common Fitness Centre,5th Most Common Fitness Centre
0,Berczy Park,Gym,Gym / Fitness Center,Gym Pool,Yoga Studio,Pilates Studio
1,"Brockton, Parkdale Village, Exhibition Place",Gym,Gym / Fitness Center,Yoga Studio,Martial Arts Dojo,Boxing Gym
2,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Gym,Yoga Studio,Sports Club,Farmers Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Gym,Gym / Fitness Center,Track,Sports Club,Pilates Studio
4,Central Bay Street,Gym,Gym / Fitness Center,Yoga Studio,Gym Pool,Bike Shop
5,Christie,Gym / Fitness Center,Yoga Studio,Martial Arts Dojo,Gym,Pilates Studio
6,Church and Wellesley,Gym,Gym / Fitness Center,Gym Pool,Yoga Studio,Pilates Studio
7,"Commerce Court, Victoria Hotel",Gym,Gym / Fitness Center,Pilates Studio,Track Stadium,Gym Pool
8,Davisville,Gym,Yoga Studio,Gym / Fitness Center,Martial Arts Dojo,Track
9,Davisville North,Gym,Yoga Studio,Gym / Fitness Center,Pilates Studio,Track


### Clustering Neighborhoods

#### Using K-Means Clustering to cluster the neighborhood into 5 clusters.

In [119]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 3, 1, 0, 0, 1, 2, 0, 2, 2], dtype=int32)

#### Create a dataframe including the cluster as well as the top 5 fitness centres for each neighborhood

In [120]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df_tor_filter

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Fitness Centre,2nd Most Common Fitness Centre,3rd Most Common Fitness Centre,4th Most Common Fitness Centre,5th Most Common Fitness Centre
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Yoga Studio,Martial Arts Dojo,Gym,Pilates Studio,College Gym
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Gym / Fitness Center,Gym,Martial Arts Dojo,Yoga Studio,Pilates Studio
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Gym / Fitness Center,Yoga Studio,Gym,Sports Club,Bike Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Gym / Fitness Center,Gym,Yoga Studio,Martial Arts Dojo,Gym Pool
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Gym / Fitness Center,Gym,College Gym,Gym Pool,Bike Shop


In [121]:
tor_merged.shape

(39, 11)

In [130]:
tor_merged.columns.values

array(['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude',
       'Cluster Labels', '1st Most Common Fitness Centre',
       '2nd Most Common Fitness Centre', '3rd Most Common Fitness Centre',
       '4th Most Common Fitness Centre', '5th Most Common Fitness Centre'],
      dtype=object)

### Identifying locations without a Yoga Studio in the Top 5 Common Fitness Centres

In [156]:
no_yoga_list = pd.DataFrame(columns=['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude',
       'Cluster Labels','1st Most Common Fitness Centre',
       '2nd Most Common Fitness Centre', '3rd Most Common Fitness Centre',
       '4th Most Common Fitness Centre', '5th Most Common Fitness Centre'])
k=0

for i in range (0,39):
    if 'Yoga Studio' not in tor_merged.loc[i].values:
        no_yoga_list.loc[k] = tor_merged.loc[i]
        k+=1
    
   
no_yoga_list

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Fitness Centre,2nd Most Common Fitness Centre,3rd Most Common Fitness Centre,4th Most Common Fitness Centre,5th Most Common Fitness Centre
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Gym / Fitness Center,Gym,College Gym,Gym Pool,Bike Shop
1,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,0,Gym,Gym / Fitness Center,Pilates Studio,Track Stadium,Gym Pool
2,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,0,Gym,Gym / Fitness Center,Pilates Studio,Track Stadium,Gym Pool
3,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Gym,Gym / Fitness Center,Martial Arts Dojo,Pilates Studio,Gym Pool
4,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,0,Gym,Gym / Fitness Center,Track,Sports Club,Pilates Studio


### Visualize the resulting locations

In [158]:
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium # map rendering

map_final = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(
        no_yoga_list['Latitude'], 
        no_yoga_list['Longitude'], 
        no_yoga_list['Borough'], 
        no_yoga_list['Neighborhood']):
    
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_final)  

map_final

## Conclusion

Five Locations have been identified for launching the Yoga Studio venture, out of which 03 are in Downtown Toronto and 02 in Central Toronto.

The exact locations are as indicated below:

In [160]:
no_yoga_list[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
2,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
3,M5N,Central Toronto,Roselawn,43.711695,-79.416936
4,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
